In [77]:
import glob, json, pathlib
from tqdm.notebook import tqdm
sanitychecks = list(glob.glob(r"D:\ig_pipeline\cad\*\*\artifacts\sanitycheck.json"))
jsons = {}
for x in tqdm(sanitychecks):
    with open(x, "r") as f:
        name = "/".join(pathlib.Path(x).parts[-4:-2])
        jsons[name] = json.load(f)

  0%|          | 0/92 [00:00<?, ?it/s]

In [78]:
total = len(jsons)
failed = {name for name, x in jsons.items() if x["errors"]}
total, len(failed)

(92, 30)

In [80]:
from collections import defaultdict
import re
error_templates_and_ignore = {
    "Category (.*) for object .* does not exist on spreadsheet.": False,
    "(.*) has too many vertices: .* > .*": True,
    "(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.": False,
    "(.*) is missing in instance .*, so relative transform check cannot be completed.": False,
    "(.*) has bad name.": False,
    "(.*) has negative object offset scale.*": False,
    "(.*) has scale that is not 1. Reset scale.": False,
    ".*No instance ID 0 instance of (.*)": False,
    "Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.": True,
    "All instances of (.*) do not have contiguous instance IDs. Missing: .*": False,
    "(.*) has disallowed type.*": False,
    "Light object (.*) should not have object offset rotation. Reset pivot.": False,
    "(.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.": False,
    "(.*) has different pivot offset rotation \(by (.*)\). Match pivots on each instance.": False,
    "(.*) has different shear. Match scaling axes on each instance.": False,
    "Model ID (.*) contains 'todo'.": False,
    "(.*) should not have an upper side.": False,
    "(.*) should have an upper side.": False,
    "(.*) has different vertex count than recorded in provider.*": False,
    "(.*) has different face count than recorded in provider.*": False,
    "(.*) has no collision mesh. Create a collision mesh.": False,
    "Inconsistent link sets within model (.*)": False,
    "(.*) is missing meta link: fillable.": False,
    "(.*) is missing meta link: heatsource.": False,
    "(.*) is missing meta link: togglebutton.": False,
    "(.*) is missing meta link: particleapplier.": False,
    "(.*) is missing meta link: fluidsource.": False,
    "(.*) is missing meta link: fluidsink.": False,
    "(.*) is missing meta link: togglebutton.": False,
    "(.*) element .* has too many vertices .*": True,
    "Model ID (.*) requires joints but has no joints.": False,
    "Expected meta links for (.*) are missing: .*": False,
    "Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*": False,
    "Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements.": False,
    "Cannot validate clothness: category (.*) not found in taxonomy.": False,
    "(.*) element .* is not a volume": True,
    "(.*) element .* has elements trimesh still finds splittable": False,
    "Object (.*) has .* fillable meshes. Should have no more than one.": False,
    "Model (.*) has category .* that is neither the from or to element in the rename file.": False,
    "(.*) has meta link not required by its synset: .*": True,
    "(.*) meta type .* ID .* has non-continuous subids .*": False,
    "Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.": False,
    "(.*) is in the deletion queue. Delete the object.": False,
    "(.*) has unapplied rename .*.": True,
    "fillable_seed is an invalid meta link under (.*).": False,
    "(.*) is not unique.": False,
    "Object (.*) has .* meshes. Should have no more than one.": False,
    "(.*) is not under another object but contains part tags {'connectedpart'}.": False,
    "(.*) is not rendering the baked material. Select the baked material for rendering or rebake.": True,
    "(.*) has non-shell material. Run texture baking.": True,
    "(.*) is not rendering the baked material in the viewport. Select the baked material for viewport.": True,
    "(.*) has different material. Match materials on each instance.": True,
    "(.*) has different UV unwrapping than recorded. Reunwrap the object.": True,
}

SKIP_IGNORE = True

error_templates = list(error_templates_and_ignore.keys())
error_matches = defaultdict(set)
error_matches_by_file = defaultdict(lambda: defaultdict(set))
unknown_errors_by_file = defaultdict(set)
for name, f in jsons.items():
    for error in f["errors"] + f["warnings"]:
        for i, tmpl in enumerate(error_templates):
            match = re.fullmatch(tmpl, error, re.S)
            if match is not None:
                if not (SKIP_IGNORE and error_templates_and_ignore[tmpl]):
                    error_matches[tmpl].add(match)
                    error_matches_by_file[tmpl][name].add(match)

                break
        else:
            print(error)
            unknown_errors_by_file[name].add(error)
        
print("\nCounts")
for tmpl, objs in sorted(error_matches.items(), key=lambda x: len(x[1]), reverse=True):
    print(f"{tmpl}: {len(objs)}")

print("\n\nCounts by file")
for tmpl, matches_by_file in error_matches_by_file.items():
    all_matches = error_matches[tmpl]
    if not all_matches:
        continue
    print(f"\n{tmpl}")
    for name, matches in sorted(matches_by_file.items(), key=lambda x: x[1], reverse=True):
        if not matches:
            break
        print(f"  {name}: {len(matches)}")

L-haystack-lujosc-0-Mcollision is a meta link at root level. Place it under the appropriate object.
potato-teqeai-0-Mcollision is a meta link at root level. Place it under the appropriate object.
sliced_lemon-andair-0-Mcollision is a meta link at root level. Place it under the appropriate object.
sliced_lemon-gcjjjt-0-Mcollision is a meta link at root level. Place it under the appropriate object.
sliced_lemon-uvncuu-0-Mcollision is a meta link at root level. Place it under the appropriate object.
L-vase-dxnzuk-0-Mcollision is a meta link at root level. Place it under the appropriate object.
salad_bowl-dhdhul-0-base_link-Mcollision is a meta link at root level. Place it under the appropriate object.
clove-usxktf-0-Mcollision is a meta link at root level. Place it under the appropriate object.
diced__beef_roast-vrzhwc-0-Mcollision is a meta link at root level. Place it under the appropriate object.
plate-efkgcw-0-Mcollision is a meta link at root level. Place it under the appropriate obj

In [81]:
from IPython.display import display, Markdown

# Print the number of kinds of error for each target
output = []
output.append("\n\n# Counts by error")
for name in jsons.keys():
    all_errors = {}
    for tmpl, matches_by_file in error_matches_by_file.items():
      if error_templates_and_ignore[tmpl]:
          continue
      matches = matches_by_file[name]
      if not matches:
          continue
      all_errors[tmpl] = matches

    for error in unknown_errors_by_file[name]:
        all_errors["Unknown"] = all_errors.get("Unknown", set())
        all_errors["Unknown"].add(error)

    if all_errors:
        output.append(f"\n## {name}")
        for k, v in sorted(all_errors.items(), key=lambda x: len(x[1]), reverse=True):
            output.append(f"  ### {k}: {len(v)}")
            for match in v:
                err_str = match if isinstance(match, str) else match.group(0)
                output.append("    " + err_str)
            output.append("")

display(Markdown("\n".join(output)))



# Counts by error

## objects/batch-01
  ### (.*) has no collision mesh. Create a collision mesh.: 3
    L-vest-girtqm-0 has no collision mesh. Create a collision mesh.
    L-tank_top-nobisg-0 has no collision mesh. Create a collision mesh.
    L-pants-tnirgd-0 has no collision mesh. Create a collision mesh.


## objects/batch-02
  ### (.*) has no collision mesh. Create a collision mesh.: 6
    L-tarp-beaglk-0 has no collision mesh. Create a collision mesh.
    L-tarp-hijmda-0 has no collision mesh. Create a collision mesh.
    L-tarp-jpawpo-0 has no collision mesh. Create a collision mesh.
    L-tablecloth-sgmepr-0 has no collision mesh. Create a collision mesh.
    L-sweatshirt-nowqqh-0 has no collision mesh. Create a collision mesh.
    L-tablecloth-shgmwu-0 has no collision mesh. Create a collision mesh.


## objects/batch-03
  ### (.*) has no collision mesh. Create a collision mesh.: 2
    bandana-wbhliu-0 has no collision mesh. Create a collision mesh.
    cardigan-itrkhr-0 has no collision mesh. Create a collision mesh.

  ### Object (.*) has .* meshes. Should have no more than one.: 1
    Object national_flag-akeryo-0 has 3 collision meshes. Should have no more than one.


## objects/batch-04
  ### (.*) has no collision mesh. Create a collision mesh.: 2
    L-wrapping_paper-hjbesb-0 has no collision mesh. Create a collision mesh.
    L-half_wrapping_paper-pjcxni-0-Tsubpart has no collision mesh. Create a collision mesh.


## objects/batch-05
  ### (.*) has no collision mesh. Create a collision mesh.: 2
    L-pennant-tfnwti-0 has no collision mesh. Create a collision mesh.
    L-quilt-xewyiz-0 has no collision mesh. Create a collision mesh.


## objects/batch-06
  ### Unknown: 1
    L-haystack-lujosc-0-Mcollision is a meta link at root level. Place it under the appropriate object.


## objects/batch-07
  ### (.*) has no collision mesh. Create a collision mesh.: 3
    L-diaper-faedft-0 has no collision mesh. Create a collision mesh.
    L-pillowcase-yakvci-0 has no collision mesh. Create a collision mesh.
    apron-uxgjdv-0 has no collision mesh. Create a collision mesh.

  ### Unknown: 1
    potato-teqeai-0-Mcollision is a meta link at root level. Place it under the appropriate object.


## objects/batch-08
  ### (.*) has no collision mesh. Create a collision mesh.: 2
    L-raincoat-wczokr-0 has no collision mesh. Create a collision mesh.
    L-sock-vpafgj-0 has no collision mesh. Create a collision mesh.


## objects/batch-09
  ### Unknown: 4
    sliced_lemon-andair-0-Mcollision is a meta link at root level. Place it under the appropriate object.
    sliced_lemon-uvncuu-0-Mcollision is a meta link at root level. Place it under the appropriate object.
    sliced_lemon-gcjjjt-0-Mcollision is a meta link at root level. Place it under the appropriate object.
    L-vase-dxnzuk-0-Mcollision is a meta link at root level. Place it under the appropriate object.

  ### (.*) has no collision mesh. Create a collision mesh.: 2
    leotard-ededis-0 has no collision mesh. Create a collision mesh.
    legging-izafif-0 has no collision mesh. Create a collision mesh.


## objects/batch-12
  ### (.*) has no collision mesh. Create a collision mesh.: 1
    L-ceremonial_robe-stcrad-0 has no collision mesh. Create a collision mesh.


## objects/legacy_batch-08
  ### (.*) has no collision mesh. Create a collision mesh.: 2
    rubber_glove-rlzvry-0-base_link has no collision mesh. Create a collision mesh.
    rubber_glove-leuiso-0-base_link has no collision mesh. Create a collision mesh.

  ### Object (.*) has .* meshes. Should have no more than one.: 1
    Object rubber_glove-plffer-0-base_link has 3 collision meshes. Should have no more than one.


## objects/legacy_batch-09
  ### (.*) has no collision mesh. Create a collision mesh.: 2
    jeans-nmvvil-0-base_link has no collision mesh. Create a collision mesh.
    jeans-pvzxyp-0-base_link has no collision mesh. Create a collision mesh.


## objects/legacy_batch-10
  ### (.*) has no collision mesh. Create a collision mesh.: 1
    dress-gtghon-0-base_link has no collision mesh. Create a collision mesh.


## objects/legacy_batch-11
  ### (.*) has no collision mesh. Create a collision mesh.: 1
    pajamas-rcgdde-0-base_link has no collision mesh. Create a collision mesh.

  ### Unknown: 1
    salad_bowl-dhdhul-0-base_link-Mcollision is a meta link at root level. Place it under the appropriate object.


## objects/legacy_batch-12
  ### (.*) has no collision mesh. Create a collision mesh.: 2
    boxers-uwzqxm-0-base_link has no collision mesh. Create a collision mesh.
    t_shirt-kvidcx-0-base_link has no collision mesh. Create a collision mesh.


## objects/substances-02
  ### Unknown: 2
    clove-usxktf-0-Mcollision is a meta link at root level. Place it under the appropriate object.
    diced__beef_roast-vrzhwc-0-Mcollision is a meta link at root level. Place it under the appropriate object.


## objects/task_relavant-ab
  ### Unknown: 1
    plate-efkgcw-0-Mcollision is a meta link at root level. Place it under the appropriate object.


## objects/task_relevant-xy
  ### (.*) has no collision mesh. Create a collision mesh.: 2
    sleeping_bag-uxqrod-0 has no collision mesh. Create a collision mesh.
    ribbon-apyxhw-0 has no collision mesh. Create a collision mesh.


## scenes/grocery_store_convenience
  ### Unknown: 1
    C-kebab-cewhbv-0-Mcollision is a meta link at root level. Place it under the appropriate object.


## scenes/hotel_suite_large
  ### (.*) has no collision mesh. Create a collision mesh.: 1
    C-blanket-uksirc-0 has no collision mesh. Create a collision mesh.


## scenes/house_double_floor_upper
  ### Unknown: 4
    Cloth object curtain-zmodhv-0 is fixed or loose. It should always be clutter.
    walls-ttopqj-0-Mcollision is a meta link at root level. Place it under the appropriate object.
    C-laptop-ongghk-0-Mcollision is a meta link at root level. Place it under the appropriate object.
    Cloth object curtain-didlbp-0 is fixed or loose. It should always be clutter.


## scenes/house_single_floor
  ### (.*) has no collision mesh. Create a collision mesh.: 3
    C-quilt-getwzm-0 has no collision mesh. Create a collision mesh.
    C-kid_glove-njidcf-0 has no collision mesh. Create a collision mesh.
    C-blanket-dnypbe-0 has no collision mesh. Create a collision mesh.


## scenes/restaurant_asian
  ### (.*) has no collision mesh. Create a collision mesh.: 1
    C-dinner_napkin-etxjzg-0 has no collision mesh. Create a collision mesh.


## scenes/school_biology
  ### Unknown: 1
    walls-ktqxej-0-Mcollision is a meta link at root level. Place it under the appropriate object.


In [59]:
from collections import defaultdict
import re

warning_re = re.compile(r"(.*) has too many vertices: (\d+) > 20000")
warn_objs = defaultdict(dict)
for name, x in jsons.items():
    for w in x["warnings"]:
        m = warning_re.fullmatch(w)
        if m:
            warn_objs[name][m.group(1)] = int(m.group(2))
            
print(warn_objs)

defaultdict(<class 'dict'>, {'objects/batch-00': {'L-tag-ohegnp-0': 21794, 'chicken_wire-czoiqw-0': 52305}, 'objects/batch-01': {'L-shoe_rack-spdrzb-0': 59456}, 'objects/batch-02': {'L-radio-vgioak-0': 50123}, 'objects/batch-05': {'L-punching_bag-svkdji-0': 20027}, 'objects/batch-06': {'zipper-qwsvid-0': 27426}, 'objects/legacy_batch-01': {'swivel_chair-fqbnuk-0-base_link': 36405, 'swivel_chair-lizuow-0-base_link': 25743, 'swivel_chair-lxupys-0-base_link': 35465, 'swivel_chair-mqxrim-0-base_link': 21060, 'swivel_chair-opkoam-0-base_link': 38343}, 'objects/legacy_batch-02': {'table_lamp-pmekkr-0-arm-body-R-lower': 23529, 'table_lamp-pmekkr-0-head-neck-R-lower': 44331, 'table_lamp-pmekkr-0-arm-body-R-upper': 23529, 'table_lamp-pmekkr-0-head-neck-R-upper': 44331, 'table_lamp-pxdhzm-0-head-arm-R-lower': 33474, 'table_lamp-pxdhzm-0-head-arm-R-upper': 33474}, 'objects/legacy_batch-03': {'desk-egzdgz-0-base_link': 20943, 'dishwasher-znfvmj-0-link_0-base_link-P-lower': 31812, 'dishwasher-znfvm

In [46]:
# Worst offenders
sorted((cnt, obj) for warn_target in warn_objs.values() for obj, cnt in warn_target.items())

[(20082, 'barbecue_sauce_bottle-gfxrnj-0'),
 (20082, 'catsup_bottle-qfvqfm-0'),
 (20082, 'chocolate_sauce_bottle-yegrkf-0'),
 (20082, 'glaze_bottle-zdxagk-0'),
 (20082, 'hot_sauce_bottle-qvpthd-0'),
 (20082, 'mustard_bottle-lgxfyv-0'),
 (20082, 'salsa_bottle-kydilb-0'),
 (20082, 'soy_sauce_bottle-saujjl-0'),
 (20082, 'squeeze_bottle-uzmhdn-0'),
 (20082, 'sugar_syrup_bottle-kdlbbq-0'),
 (20082, 'vanilla_bottle-drevku-0'),
 (20082, 'white_sauce_bottle-gtwngf-0'),
 (20082, 'wine_sauce_bottle-vqtevv-0'),
 (20279, 'pot_plant-kxmvco-0-base_link'),
 (20943, 'desk-egzdgz-0-base_link'),
 (21060, 'swivel_chair-mqxrim-0-base_link'),
 (21794, 'L-tag-ohegnp-0'),
 (22097, 'L-garden_chair-deuron-0'),
 (22097, 'L-garden_chair-deuron-1'),
 (22560, 'room_light-fxtqtn-0'),
 (22560, 'room_light-fxtqtn-1'),
 (22740, 'fridge-jtqazu-0-base_link'),
 (22956, 'L-pedestal_table-ifmhpn-0'),
 (22956, 'L-pedestal_table-ifmhpn-1'),
 (22998, 'C-stockpot-gxiqbw-0'),
 (22998, 'C-stockpot-gxiqbw-1'),
 (23465, 'floor_lam

In [47]:
# Check which of the "articulated object has diff scale" errors are on bad objects
providers = json.loads(pathlib.Path(r"D:\ig_pipeline\artifacts\pipeline\object_inventory.json").read_text())["providers"]
id_providers = {k.split("-")[-1]: v for k, v in providers.items()}
categories = dict([tuple(reversed(k.split("-"))) for k in providers.keys()])
articulation_tmpl = "Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script."
for target, objs in error_matches_by_file[articulation_tmpl].items():
    if not objs:
        continue
    bad_objs = set()
    for match in objs:
        obj = match.group(1)
        if id_providers[obj] != target:
            name = f"{categories[obj]}-{obj}"
            bad_objs.add(name)

    if not bad_objs:
        continue
    
    print(f"{target}: {len(bad_objs)} {bad_objs}")

scenes/Beechwood_0_garden: 6 {'top_cabinet-dmwxyl', 'bottom_cabinet_no_top-qohxjq', 'bottom_cabinet_no_top-spojpj', 'bottom_cabinet-immwzb', 'window-ulnafj', 'door-lvgliq'}
scenes/Beechwood_0_int: 7 {'top_cabinet-dmwxyl', 'bottom_cabinet_no_top-qohxjq', 'bottom_cabinet_no_top-spojpj', 'bottom_cabinet-immwzb', 'window-ulnafj', 'door-ohagsq', 'door-lvgliq'}
scenes/Beechwood_1_int: 9 {'top_cabinet-dmwxyl', 'bottom_cabinet_no_top-pluwfl', 'window-ithrgo', 'bottom_cabinet-jhymlr', 'toilet-kfmkbm', 'bottom_cabinet-kubcdk', 'window-ulnafj', 'door-ohagsq', 'door-lvgliq'}
scenes/Benevolence_0_int: 1 {'door-lvgliq'}
scenes/Benevolence_1_int: 6 {'bottom_cabinet_no_top-ufhpbn', 'top_cabinet-dmwxyl', 'top_cabinet-eobsmt', 'top_cabinet-lsyzkh', 'window-ulnafj', 'cedar_chest-fwstpx'}
scenes/Benevolence_2_int: 2 {'window-ithrgo', 'door-lvgliq'}
scenes/hotel_gym_spa: 2 {'door-ceddpg', 'door-nfrbch'}
scenes/house_single_floor: 1 {'bottom_cabinet_no_top-gjrero'}
scenes/Ihlen_0_int: 3 {'door-ktydvs', 'win

In [48]:
# Get a list of all the objects that show up in the match links relevant errors
mismatched_link_offset_tmpl = "(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*."
objects_for_match_links = error_matches[mismatched_link_offset_tmpl] | error_matches[articulation_tmpl]
print(", ".join(f'"{x.group(1)}"' for x in sorted(objects_for_match_links, key=lambda x: x.group(1))))

"bamfsz", "bamfsz", "bamfsz", "bamfsz", "bmsclc", "ceddpg", "ceddpg", "ceddpg", "dladgw", "dladgw", "dladgw", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dnyzym", "dqnbsj", "dqnbsj", "ehwmol", "ehwmol", "emeeke", "emeeke", "eobsmt", "fexqbj", "fexqbj", "fexqbj", "fqhdne", "fqhdne", "fqhdne", "fwstpx", "fwstpx", "gjrero", "gpkbiw", "gpkbiw", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "immwzb", "immwzb", "immwzb", "inmymj", "inmymj", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "jhymlr", "jhymlr", "jhymlr", "jrhgeu", "kfmkbm", "kfmkbm", "kfmkbm", "ktydvs", "ktydvs", "ktydvs", "kubcdk", "kwbnhy", "kwbnhy", "kwbnhy", "kwbnhy", "kwbnhy", "lcrefl", "lsyzkh", "lsyzkh", "lsyzkh", "lsyzkh", "lsyzkh", "lsyzkh", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", 

In [49]:
import sys
sys.path.append(r"D:\ig_pipeline")
from b1k_pipeline.utils import parse_name

# Get a link of all the objects that are bad cloth
cloth_tmpl = "Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements."
objects_for_cloth = [parse_name(x.group(1)).group('model_id') for x in error_matches[cloth_tmpl]]
print(", ".join(f'"{x}"' for x in sorted(objects_for_cloth)))

"agftpm", "aiftuk", "ajcjzr", "bgvwes", "bpjqwj", "bpjqwj", "bpjqwj", "bpjqwj", "brebdk", "crbvcg", "cuoayr", "dnypbe", "efmfdf", "ekcjci", "eknmzl", "faedft", "feohwy", "fglfga", "ficvss", "fmapwe", "fzldgi", "getwzm", "girtqm", "gtghon", "gxmcpy", "htcikj", "imdhfp", "itrkhr", "ivmsdt", "jfsovh", "jkcugp", "jmujjo", "kclcrj", "kiiium", "klhkgd", "laksie", "ldpbcu", "lqweda", "mksdlu", "mpxtgf", "mxxyva", "njidcf", "nmvvil", "nogevo", "nowqqh", "nzbhjq", "ohvomi", "ohvomi", "ojgwec", "omsbux", "padfds", "pbytey", "pchzob", "prhems", "pvzxyp", "qewdqa", "qofxar", "rcgdde", "remcyk", "rhgweu", "rygqoc", "sehjcp", "shbakk", "shbakk", "stcrad", "sullco", "szdonp", "szdonp", "szdonp", "szdonp", "tfnwti", "thmepr", "tnirgd", "trwest", "ttgike", "uaidmc", "uksirc", "uksirc", "uwzqxm", "uxgjdv", "vfknnl", "vpafgj", "vqbvph", "wbhliu", "wczokr", "xeskbh", "xzuywj", "ydgivr", "ykiiqr", "ylycdz"
